<a href="https://colab.research.google.com/github/gabrielnichio/NN-training/blob/main/Training_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

## Alocating device to use CPU

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


# Classification

## Using sklearn wine dataset

In [ ]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


## Defining NN

Inner layers with ReLU activation. Using softmax to give probability distribution on the output.

In [ ]:
class WineClassifier(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.Softmax()

  def forward(self, X):
    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output

In [ ]:
input_size = data.shape[1]
hidden_size = 32
output_size = len(wine.target_names)

wine_net = WineClassifier(input_size, hidden_size, output_size).to(device)

wine_net = wine_net.to(device)

In [ ]:
print(wine_net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


## Loss function

Using Cross Entropy Loss.

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype, Ytns.dtype)

torch.float32 torch.int64


In [ ]:
pred = wine_net(Xtns)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


 The predictions and actual values may have different dimension when using Cross Entropy Loss for CLASSIFICATION.

In [ ]:
print(pred.shape, Ytns.shape)

torch.Size([178, 3]) torch.Size([178])


In [ ]:
loss = criterion(pred, Ytns)
print(loss)

tensor(1.2200, device='cuda:0', grad_fn=<NllLossBackward0>)


# Regression

# Using sklearn diabetes dataset

In [ ]:
diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

In [ ]:
print(data[0])
print(target[0])

[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990749 -0.01764613]
151.0


In [ ]:
input_size = data.shape[1]
hidden_size = 32
output_size = 1

net = WineClassifier(input_size, hidden_size, output_size).to(device)

## Loss function
Using MSE Loss.

Predictions and actual values must have same dimensions when using MSE Loss for REGRESSION.

In [ ]:
criterion = nn.MSELoss().to(device)

Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(Xtns.shape, Ytns.shape)

torch.Size([442, 10]) torch.Size([442])


In [ ]:
pred = net(Xtns)

loss = criterion(pred.squeeze(), Ytns)
print(loss.data)

tensor(28771.2168, device='cuda:0')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Using L1 Loss

In [ ]:
criterion = nn.L1Loss().to(device)

lossL1 = criterion(pred.squeeze(), Ytns)
print(lossL1.data)

tensor(151.1335, device='cuda:0')
